In [58]:
import os
import torch
import torchvision
from glob import glob
from PIL.Image import open as im_open
import subprocess
import json
import pandas as pd

In [59]:
mode="train"
data_type="mordern"

In [60]:
src=os.path.join("/workspace", mode, data_type)
label_paths = os.path.join(src, "labels")
annotation_paths = os.path.join(src, "annotations")
img_paths = os.path.join(src, "images")

In [61]:
labels = list(glob(os.path.join(label_paths, "*.txt")))
annotations = list(glob(os.path.join(annotation_paths, "*.json")))
imgs = list(glob(os.path.join(img_paths, "*.jpg")))

In [62]:
for img_path, ann_path, label_path in zip(imgs, annotations, labels):
    break
    img = im_open(img_path).convert("RGB")
    transoforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomGrayscale(),
        torchvision.transforms.GaussianBlur(kernel_size=7),
        torchvision.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        torchvision.transforms.ToPILImage(),        
    ])
    img = transoforms(img)
    img_folder = "/".join(img_path.split("/")[:-1])
    ann_folder = "/".join(ann_path.split("/")[:-1])
    label_folder = "/".join(label_path.split("/")[:-1])
    name = img_path.split("/")[-1].split(".")[0]
    dest = img_folder + "/" + name + "_2" + ".jpg"
    with open(dest, "w") as f:
        img.save(f)
    dest = ann_folder + "/" + name + "_2" + ".json"
    subprocess.run(f"cp {ann_path} {dest}", shell=True)
    dest = label_folder + "/" + name + "_2" + ".txt"
    subprocess.run(f"cp {label_path} {dest}", shell=True)

In [63]:
for img_path, ann_path, label_path in zip(imgs, annotations, labels):    
    img_folder = "/".join(img_path.split("/")[:-1])
    ann_folder = "/".join(ann_path.split("/")[:-1])
    label_folder = "/".join(label_path.split("/")[:-1])
    name = img_path.split("/")[-1].split(".")[0]
    dest = img_folder + "/" + name + "_" + ".jpg"
    if os.path.exists(dest):
        continue
    df = pd.read_csv(label_path, sep=" ", header=None)
    df = df[df.loc[:, 0].isin([7, 6] if data_type == "mordern" else [5])]
    if df.empty:
        continue
    img = im_open(img_path).convert("RGB")
    transoforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomGrayscale(),
        torchvision.transforms.GaussianBlur(kernel_size=7),
        torchvision.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        torchvision.transforms.ToPILImage(),        
    ])
    img = transoforms(img)    
    with open(dest, "w") as f:
        img.save(f)
    dest = ann_folder + "/" + name + "_" + ".json"
    subprocess.run(f"cp {ann_path} {dest}", shell=True)
    dest = label_folder + "/" + name + "_" + ".txt"
    subprocess.run(f"cp {label_path} {dest}", shell=True)